In [ ]:
!pip install numpy scikit-learn

In [10]:
from sklearn.datasets import make_regression
import numpy as np

In [33]:
import time
import requests

BASE_URL = "http://localhost:8000"

def train_and_wait_for_completion(model_name, model_type, X, y, config=None, check_interval=5):
    """
    Запускает обучение модели и ожидает его завершения.
    """
    payload = {
        "model_name": model_name,
        "model_type": model_type,
        "X": X,
        "y": y,
        "config": config or {"params": {}},
    }
    response = requests.post(f"{BASE_URL}/fit", json=payload)
    
    if response.status_code != 200:
        raise RuntimeError(f"Ошибка запуска обучения: {response.json()}")

    print(f"Обучение модели `{model_name}` запущено. Ожидание завершения...")
    
    while True:
        status_response = requests.get(f"{BASE_URL}/status", params={"model_name": model_name})
        
        status_data = status_response.json()
        print(f"Статус: {status_data['message']}")
        
        if "завершено" in status_data["message"]:
            break
        elif "ошибка" in status_data["message"]:
            raise RuntimeError(f"Обучение модели `{model_name}` завершилось с ошибкой: {status_data['message']}")
        
        time.sleep(check_interval)

In [34]:
X = np.random.rand(1_000_000, 10).tolist()
y = np.random.rand(1_000_000).tolist()

In [35]:
%time
train_and_wait_for_completion(
            model_name="test_model1",
            model_type="LinearRegression",
            X=X,
            y=y,
            config={},
        )

CPU times: total: 0 ns
Wall time: 0 ns
Обучение модели `test_model1` запущено. Ожидание завершения...
Статус: Задача обучения для модели `test_model1` не найдена.
Статус: Задача обучения для модели `test_model1` не найдена.
Статус: Задача обучения для модели `test_model1` не найдена.
Статус: Задача обучения для модели `test_model1` не найдена.
Статус: Задача обучения для модели `test_model1` не найдена.


KeyboardInterrupt: 